# 5: Bioengineering
## This notebook shows the code and calculations used to determine:
### - Biomass macromolecule reductant cost (Table 3)
### - Overproduction of hexadecanoate, isopentenyl PP, and chorismate (Fig. 5)
### - Production versus inoculation density and photon flux (Fig. 6)


In [ ]:
import pandas as pd
import numpy as np
import csv
import cobra
from cobra.core.metabolite import elements_and_molecular_weights
elements_and_molecular_weights['R']=0.0
elements_and_molecular_weights['Z']=0.0
from matplotlib import pyplot as plt
import seaborn as sns
sns.set()
import fcns.modeling_functions as fcns

## Table 3: %LET, reductant cost and theoretical yield of biomass macromolecules

In [ ]:
## Calculated for high light acclimated cells during the last simulation interval
## Get a solution for the HL model 
base_model = cobra.io.load_json_model('HL_base_iLB1035.json')
(yII_biomass, yII_model) = fcns.simulate(base_model,'HL',photon_const=True,Po_const=True,
                                         YII_const=True,D1_const=True,DM20=False,ngam_comp='mito')


In [ ]:
## The total biomass at the start of the simulation is in yII_biomass as time point 700 min
## The accumulated biomass is the difference between that value and the value at 720 min
mgDW = np.around(yII_biomass['700']['Biomass'],3)
DW_accum = np.around(yII_biomass['720']['Biomass']-yII_biomass['700']['Biomass'],3)

# Column 1: Percent Biomass
## The percent biomass for each macromolecule is encoded in the biomass objective function
bof_dict = {}
labels = {'biomass_carb_c':'Structural carb',
          'biomass_dna_c': 'DNA',
          'biomass_mem_lipids_c': 'Membrane lipids',
          'biomass_pigm_h':'Pigments',
          'biomass_plastid_lip_h':'Plastid lipids',
          'biomass_pro_c':'Protein',
          'biomass_rna_c':'RNA',
          'carbon_storage_c':'Storage'}

for m in yII_model.reactions.bof_c.metabolites:
    if yII_model.reactions.bof_c.metabolites[m] < 0.:
        label = labels[m.id]
        bof_dict[label]=np.around(-100*yII_model.reactions.bof_c.metabolites[m],1)       

# Column 2: %LET
## Get the baseline LET
base_LET = np.around(2*yII_model.reactions.PSI_u.flux-yII_model.reactions.CEF_h.flux,1)
### Units are mmol e- per simulation time interval (20 min)

## Determine the LET for biomass only, no EET
hv_unconstrained = yII_model.copy()
hv_unconstrained.reactions.DM_photon_c.upper_bound = 1e5
hv_unconstrained.objective = 'DM_photon_c'

hv_unconstrained.reactions.bof_c.lower_bound = DW_accum*0.999

solution = cobra.flux_analysis.parsimonious.pfba(hv_unconstrained)
bof_LET = np.around(2*hv_unconstrained.reactions.PSI_u.flux-hv_unconstrained.reactions.CEF_h.flux,1)

### The LET for each component is determined by:
#### 1) Removing the component from the biomass objective function
#### 2) Reducing the DW_accum by the percent of total biomass for that component
#### 3) Simulating with the new BOF and re-calculating LET
#### This new value is the number of electrons required to build all biomass components
#### EXCEPT the one under evaluation. The difference between this number and baseline is the
#### LET required for the component under investigation

#### Get LET without the component
out_var = fcns.get_comp_let(yII_model,DW_accum)

#### LET delta
per_LET = {}
total_LET = 0.
for k,v in out_var.items():
    per_LET[k] = np.around(((bof_LET-v)/base_LET)*100.,1)
    total_LET = np.around(total_LET+((bof_LET-v)/base_LET)*100.,1)

# Column 3: Reductant Cost
## Equal to the LET for a component divided by the fraction of total biomass
### units: mmol e- per gram DW of the component (e.g. protein fraction)
red_cost = {}
for k,v in per_LET.items():
    perc = bof_dict[k]/100.
    mgComp = perc*DW_accum
    met_let = v/100.*base_LET
    red_cost[k]=np.around(met_let/mgComp,0)

# Column 4: Theoretical Yield
## Equal to the milligrams dry weight of the biomass component if all EET could be routed to 
## synthesize that macromolecule
### units: mg biomass component per g dry weight total biomass per hour

EET = base_LET-bof_LET
th_yield = {}
for k,v in per_LET.items():
    met_let = v/100.*base_LET
    cost = red_cost[k]
    mg_yield = (EET/cost)/20*60/mgDW*1000.
    th_yield[k]=np.around(mg_yield,1)
    
# Make the table
table3 = pd.DataFrame(index=labels.values())
table3['Biomass percent']=bof_dict.values()
table3['%LET']=per_LET.values()
table3['Reductant Cost']=red_cost.values()
table3['Theoretical yield']=th_yield.values()

In [ ]:
table3

In [ ]:
print('EET %LET: ', np.around(EET/base_LET*100,1))
print('Other %LET', np.around(100.-(EET/base_LET*100)-total_LET,1))

## Figure 5. Flux changes due to overexpression of bioengineering targets
### This code will return the heat map used to build the figure as well as the plots of product yield versus fraction of biomass redirected



In [ ]:
# The reactions used in the figure are listed below
rxns = ['RUBISC_h','RUBISO_h','PGK_h','GAPDH_h','FBA_h','FBP_h','TKL2_h',
        'RPE_h','TPI_h','TKL1_h','RPI_h','SBP_h','FBA3_h','PGAM_h','ENO_h',
        'PYK_h','PDH_E2_h','ACCOAC_h']
## the suffix '_h' denotes the plastid localization of the reaction

## Get the baseline flux distribution from the Y(II) constrained model
soln = cobra.flux_analysis.parsimonious.pfba(yII_model)
base_line_flux = [np.around(soln.fluxes[r],3) for r in rxns]

## Build the dataframe to collect outputs
heatmap_data = pd.DataFrame(index=rxns)

# Get the fluxes for overexpression at 30% BOF redirected
mets = ['chor_h','hdca_h','ipdp_h']
frac_bof = [0.7]

for m in mets:
    met_eng = m
    temp_model = yII_model.copy()
    temp_model = fcns.target_met(temp_model,met_eng)
    for x in frac_bof:
        (out_model,fluxes)= fcns.calc_eet(temp_model, frac_max = x, met = met_eng,
                                  ref_gDW=mgDW, TI=20./60., get_fluxes = True, get_model=True)
    
    meteng_flux = [np.around(out_model.reactions.get_by_id(r).flux,3) for r in rxns]
    delta_flux = abs(np.array(meteng_flux))-(abs(np.array(base_line_flux)))
    
    ## delta_flux is in umol/TI, convert to mmol gDW-1 h-1
    delta_flux = delta_flux/mgDW/20.*60.
    
    
    heatmap_data[m]=delta_flux


In [ ]:
c_map = sns.diverging_palette(225, 30, l=50, as_cmap=True)
ax = sns.heatmap(heatmap_data, linewidth = 0.5, center=0, square=True, cmap = c_map)

In [ ]:
## Production versus fraction of redirected biomass
frac_bof = [0.9,0.8,0.7,0.6,0.5]
plot_data = pd.DataFrame(index=(1-np.array(frac_bof)))

mets = ['hdca_h','ipdp_h','chor_h']

for m in mets:
    out_yield = []    
    met_eng = m
    temp_model = yII_model.copy()
    temp_model = fcns.target_met(temp_model,met_eng)
    for x in frac_bof:
        fluxes= fcns.calc_eet(temp_model, frac_max = x, met = met_eng,
                                  ref_gDW=mgDW, TI=20./60., get_fluxes = True, get_model=False)
        r_id = 'DM_'+met_eng
        out_yield.append(fluxes[r_id]/mgDW*60./20.)
    
    plot_data[m]=out_yield

In [ ]:
plot_data

In [ ]:
fig, axs = plt.subplots(3,figsize=(5,8))
x = plot_data.index.values

axs[0].plot(x,plot_data['hdca_h'].values, linestyle = 'dashed', linewidth = 3)
axs[0].set_title("Hexadecanoate production")
axs[0].set_ylabel("mmol product gDW-1 h-1")
axs[0].set_xlabel("Fraction of biomass redirected")
axs[0].set_ylim([0.,0.3])
axs[0].set_xlim([0.,0.6])

axs[1].plot(x,plot_data['ipdp_h'].values, linestyle = (0,(5,5)), linewidth = 3)
axs[1].set_title("Isoprenoid production")
axs[1].set_ylabel("mmol product gDW-1 h-1")
axs[1].set_xlabel("Fraction of biomass redirected")
axs[1].set_ylim([0.,0.3])
axs[1].set_xlim([0.,0.6])

axs[2].plot(x,plot_data['chor_h'].values, linewidth = 3)
axs[2].set_title("Shikimate production")
axs[2].set_ylabel("mmol product gDW-1 h-1")
axs[2].set_xlabel("Fraction of biomass redirected")
axs[2].set_ylim([0.,0.3])
axs[2].set_xlim([0.,0.6])

fig.tight_layout()

## Figure 6: Production envelopes for overexpression designs at different inoculation densities

### Fig. 6 A and B

In [ ]:
base_model = cobra.io.load_json_model('HL_base_iLB1035.json')
(yII_biomass_LL, yII_model_LL) = fcns.simulate(base_model,'HL',photon_const=True,Po_const=True,
                                         YII_const=True,D1_const=True,DM20=False,ngam_comp='mito')

model = yII_model.copy()
met_eng = 'ipdp_h'
bof_frac = 0.6 #Fraction redirected
inoc_array = [100000.0, 100050000.0, 10095000.0, 110045000.0, 120040000.0, 130035000.0,
              1389495.0, 140030000.0, 150025000.0, 160020000.0, 170015000.0, 180010000.0,
              190005000.0, 193070.0, 200000000.0, 20090000.0, 2682696.0, 30085000.0,
              372759.0, 40080000.0, 50075000.0, 5179475.0, 60070000.0, 70065000.0, 719686.0,
              80060000.0, 90055000.0]

light = 'HL'
model = fcns.target_met(model,met_eng)

time_interval = 360 ## Note, sims for figure 6 used a 60 min interval,
                    ## but for demonstration we use 360 here
                    ## 60 takes longer and uses quite a bit of memory
interval = [str(a) for a in range(time_interval,7200+time_interval,time_interval)]

production_DF = pd.DataFrame(columns=interval)

for inn in inoc_array:
    temp_DF = fcns.get_prod_env(model,met_eng,bof_frac,inn,light, time_interval)
    temp_DF = temp_DF.drop(['0'],axis= 1)
    production_DF.loc[inn]=temp_DF.values[0]
    

In [ ]:
fcns.plot_prod(production_DF,time_interval,metgfw=243.07,bof_frac=bof_frac)

In [ ]:
base_model = cobra.io.load_json_model('LL_base_iLB1035.json')
(yII_biomass_LL, yII_model_LL) = fcns.simulate(base_model,'LL',photon_const=True,Po_const=True,
                                         YII_const=True,D1_const=True,DM20=False,ngam_comp='mito')

model = yII_model.copy()
met_eng = 'ipdp_h'
bof_frac = 0.6 #Fraction redirected
inoc_array = [100000.,   170125.,   242446., 345511.,   492388., 650000., 800000., 1000000.,  1193777.,
        1425103.,  1701254.,  2030918.,  2424462.,  2894266.,  3455107., 4124626., 5500000., 7000000.,
        8500000.,  10000000., 11937766., 14251027., 17012543., 20309176., 24244620., 28942661., 34551073.,
        41246264., 50000000., 60000000.]

light = 'LL'
if light == 'LL':
    model = yII_model_LL
model = fcns.target_met(model,met_eng)

time_interval = 360 ## Note, sims for figure 6 used a 60 min interval,
                    ## but for demonstration we use 360 here
                    ## 60 takes longer and uses quite a bit of memory
interval = [str(a) for a in range(time_interval,7200+time_interval,time_interval)]

production_DF = pd.DataFrame(columns=interval)

for inn in inoc_array:
    temp_DF = fcns.get_prod_env(model,met_eng,bof_frac,inn,light, time_interval)
    temp_DF = temp_DF.drop(['0'],axis= 1)
    production_DF.loc[inn]=temp_DF.values[0]


In [ ]:
fcns.plot_prod(production_DF,time_interval,metgfw=243.07,bof_frac=bof_frac)